# The Visualizer
It looks like we still can't get more than 21 fps on my computer, using Qt4Agg. 
Profiler shows that the majority of time is still spent blitting.
Maybe this means it is time to move to qtplot.

In [1]:
import pyqtgraph.examples
pyqtgraph.examples.run()

In [1]:
import numpy as np
import pyaudio
import wave
import struct
import matplotlib.pyplot as plt
import random
import scipy.signal
import scipy.io.wavfile
import time 
import matplotlib

import cProfile, pstats, io


def profile(fnc):
    
    """A decorator that uses cProfile to profile a function"""
    
    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable()
        retval = fnc(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return inner

# matplotlib.use('GTK3Agg') 
matplotlib.use('Qt4Agg')
# matplotlib.use('tkagg')

# %matplotlib tk


Bad key "text.kerning_factor" on line 4 in
/Users/tylerkeeling/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key "text.kerning_factor" on line 4 in
/Users/tylerkeeling/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
@profile
def graph(end=-1):
    global data, fig, ax, stream, line, CHUNK, background
    
    # for measuring frame rate
    frame_count = 0
    start_time = time.time()
    
    while len(data) > 0 and (end==-1 or end > frame_count):
        stream.write(data) 
        data = wf.readframes(CHUNK) # len 0 at end of wf

        data_int = struct.unpack(str(2*CHUNK)+'h', data)
        data_np = np.array(data_int, dtype=np.int8)[::2]
    
        # update figure canvas
        try:

            # Basic plt instructions
#             ax.clear()
#             ax.plot(data_np)
#             plt.pause(0.001)

            # Optimized plt? 
#             line.set_ydata(data_np)
#             fig.canvas.draw()
#             fig.canvas.flush_events()

            # Optimized 2?
            line.set_ydata(data_np)
            fig.canvas.restore_region(background)
            ax.draw_artist(line)
            fig.canvas.blit(ax.bbox)
            fig.canvas.flush_events()

            frame_count += 1


#         except Exception as e:
#             print(e)
        except:
            # calculate average frame rate
            frame_rate = frame_count / (time.time() - start_time)

            print('stream stopped')
            print('average frame rate = {:.0f} FPS'.format(frame_rate))
            break
            
            
def graph_single():
        global data, fig, ax, stream, line, frame_count, start_time, CHUNK

        stream.write(data)
        data = wf.readframes(CHUNK)

        data_int = struct.unpack(str(2*CHUNK)+'h', data)# [::2] skips every other data point 
        data = np.array(data_int, dtype=np.int8)
        
        ax.clear()
        ax.plot(data)
        plt.show(block=True)
    

In [3]:
"""PyAudio Example: Play a wave file."""

CHUNK = 1024

wf = wave.open("02-Nangs.wav", 'rb')

# instantiate PyAudio (1)
p = pyaudio.PyAudio()

# open stream (2)
stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True)

# read data
data = wf.readframes(CHUNK)

#Matplotlib
fig, ax = plt.subplots(1, figsize=(15, 7))
x = np.arange(0, 2 * CHUNK, 2)

ax.set_title('AUDIO WAVEFORM')
ax.set_xlabel('samples')
ax.set_ylabel('volume')
ax.set_ylim(-128, 128)
ax.set_xlim(0, 2 * CHUNK)

fig.canvas.draw()
background = fig.canvas.copy_from_bbox(ax.bbox)
line, = ax.plot(x, np.zeros(CHUNK), '-', lw=2)
plt.show(block=False)
plt.pause(0.1)

print('stream started')

graph()
        
# play stream (3)
stream.close()

# close PyAudio (5)
p.terminate()

stream started
stream started
stream stopped
average frame rate = 19 FPS
         52522 function calls (51751 primitive calls) in 8.145 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.012    0.012    8.145    8.145 <ipython-input-2-205eb9d71b94>:1(graph)
      152    0.002    0.000    5.203    0.034 /Users/tylerkeeling/opt/anaconda3/lib/python3.7/site-packages/matplotlib/backends/backend_qt5agg.py:80(blit)
      152    0.218    0.001    5.193    0.034 {built-in method repaint}
      122    0.016    0.000    4.975    0.041 /Users/tylerkeeling/opt/anaconda3/lib/python3.7/site-packages/matplotlib/backends/backend_qt5agg.py:23(paintEvent)
      122    3.305    0.027    4.593    0.038 /Users/tylerkeeling/opt/anaconda3/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:2124(_unmultiplied_rgba8888_to_premultiplied_argb32)
      153    0.001    0.000    2.234    0.015 /Users/tylerkeeling/opt/anaconda3/lib/pyt

stream stopped
average frame rate = 19 FPS
         52522 function calls (51751 primitive calls) in 8.145 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.012    0.012    8.145    8.145 <ipython-input-2-205eb9d71b94>:1(graph)
      152    0.002    0.000    5.203    0.034 /Users/tylerkeeling/opt/anaconda3/lib/python3.7/site-packages/matplotlib/backends/backend_qt5agg.py:80(blit)
      152    0.218    0.001    5.193    0.034 {built-in method repaint}
      122    0.016    0.000    4.975    0.041 /Users/tylerkeeling/opt/anaconda3/lib/python3.7/site-packages/matplotlib/backends/backend_qt5agg.py:23(paintEvent)
      122    3.305    0.027    4.593    0.038 /Users/tylerkeeling/opt/anaconda3/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:2124(_unmultiplied_rgba8888_to_premultiplied_argb32)
      153    0.001    0.000    2.234    0.015 /Users/tylerkeeling/opt/anaconda3/lib/python3.7/site-packages/matplotli

Thanks to the following: 
https://www.youtube.com/watch?v=AShHJdSIxkY
https://stackoverflow.com/questions/40126176/fast-live-plotting-in-matplotlib-pyplot
https://bastibe.de/2013-05-30-speeding-up-matplotlib.html
